## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [37]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [38]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000028B85AF59C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000028B85AF6530>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [39]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Jay and I am a Chief AI Engineer")])

AIMessage(content="Hi Jay, it's nice to meet you! That's a fascinating role. What kind of AI projects are you currently working on? \n\nI'm always eager to learn more about the exciting advancements happening in the field of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 22, 'total_tokens': 75, 'completion_time': 0.096363636, 'prompt_time': 0.00014241, 'queue_time': 0.00347877, 'total_time': 0.096506046}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-421b29b1-98a9-4436-aa0f-86d78ed64fe1-0', usage_metadata={'input_tokens': 22, 'output_tokens': 53, 'total_tokens': 75})

In [40]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Jay and I am a Chief AI Engineer"),
        AIMessage(content="Hello Jay! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)  

# here LLM will able to remember the chat history 

AIMessage(content="You said your name is Jay, and you are a Chief AI Engineer.  \n\nIs there anything else you'd like to tell me about yourself or your work? I'm happy to chat! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 97, 'total_tokens': 143, 'completion_time': 0.083636364, 'prompt_time': 0.002499136, 'queue_time': 0.003903913, 'total_time': 0.0861355}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-fdc3ed18-c434-42f9-a344-7be17751a531-0', usage_metadata={'input_tokens': 97, 'output_tokens': 46, 'total_tokens': 143})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [41]:
!pip install langchain_community

get_session_history():-> it is function which will able to retrive whatever chat history for specific session_id then we 
are using  RunnableWithMessageHistory so we can intract with our LLm model based on chat history . here i.e we are giving 2 parameter (model,get_session_history)

get_session_history in fun whenever we give session_id it should able to check  where the session_idis  present in particualr dictionary or not 
if it is avalible it will go and pickup entire chat msg that what ever Q we have ask to our LLM model for that session_id and 
it will return it : 


-> BaseChatMessageHistory: it is an abtract class for storing chat msg history 
-> ChatMessageHistory :it is an in memory implimentation of Chat message history store msg in the memory list 
-> RunnableWithMessageHistory : to execute the entire opearation you need it it manage Chat msg history for other runnable 


In [42]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}
    
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [43]:
config={"configurable":{"session_id":"chat1"}}

In [44]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Jay and I am a Chief AI Engineer")],
    config=config
)

In [45]:
response.content

"Hello Jay,\n\nIt's nice to meet you!  \n\nThat's a fascinating role. As a large language model, I'm always eager to learn more about how AI is being used in different fields. What kind of projects are you working on as Chief AI Engineer?\n"

In [46]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
) 


# it is able to remember it bcoz we are giving the same config :

AIMessage(content='You told me your name is Jay!  😊 \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 97, 'total_tokens': 122, 'completion_time': 0.045454545, 'prompt_time': 0.003454004, 'queue_time': 0.003650126, 'total_time': 0.048908549}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e5d5a76d-d4e6-4f84-80ee-7bc17de0da8d-0', usage_metadata={'input_tokens': 97, 'output_tokens': 25, 'total_tokens': 122})

In [47]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content 


# here it is not able to remember it bcoz we chage our seeion_id 

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to know! 😊\n"

In [48]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you. 👋  \n\nIs there anything I can help you with today?\n"

In [49]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John, as you told me a moment ago. 😊  \n\nDo you have any other questions for me? \n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [50]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [51]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Jay")]})

AIMessage(content="Hi Jay, it's nice to meet you!\n\nI'm ready to answer your questions. Just ask away! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 31, 'total_tokens': 61, 'completion_time': 0.054545455, 'prompt_time': 0.00030284, 'queue_time': 0.002618151, 'total_time': 0.054848295}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ae824e47-26bb-46ab-b60f-792d212e36a3-0', usage_metadata={'input_tokens': 31, 'output_tokens': 30, 'total_tokens': 61})

In [52]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [53]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")],
    config=config
)

response

AIMessage(content="Hi Krish, it's nice to meet you!  \n\nI'm ready to answer your questions. Ask away! 😊  What can I help you with today? \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 31, 'total_tokens': 71, 'completion_time': 0.072727273, 'prompt_time': 0.000304039, 'queue_time': 0.0027342, 'total_time': 0.073031312}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-4208503c-e656-4ff6-8a54-c35bca82ad88-0', usage_metadata={'input_tokens': 31, 'output_tokens': 40, 'total_tokens': 71})

In [54]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Krish! I remember that you told me. 😊  \n\n\n\n'

In [55]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [56]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Hindi"})
response.content

'नमस्ते कृष्ण! 😊  \n\nमुझे बहुत खुशी है कि आपने मुझसे बात करना शुरू की। आप मुझसे कुछ भी पूछ सकते हैं, मैं अपनी पूरी कोशिश करूंगा कि आपको मदद कर सकूँ। \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [57]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [58]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Jay")],"language":"Marathi"},
    config=config
)
repsonse.content

'नमस्कार Jay, मी तुमच्या मदतीसाठी येथे आहे. तुम्ही काय विचारात आहात? \n'

In [59]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Marthi"},
    config=config,
)

In [60]:
response.content

'तुमचे नाव Jay आहे.  😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [61]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

c:\Users\Jayraj\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Jayraj\anaconda3\envs\myenv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jayraj\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [40]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal preferences like your favorite ice cream flavor.  \n\nWhat's your favorite ice cream? 😊🍦\n"

In [41]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2" 😊  \n\n\n\n'

In [42]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [43]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [44]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

In [ ]:
from langchain_groq import ChatGroq 

model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key) 

model 

In [ ]:
from langchain_core.messages import HumanMessage

from langchain_community.chat_message_histories import  ChatMessageHistory 
from langchain_core.chat_history import BaseChatMessageHistory 
from langchain_core.runnables.history import RunnableWithMessageHistory 

In [ ]:
Store={} 

def get_session_history(session_id:str)-> BaseChatMessageHistory :
    if session_id not in Store :
        Store[session_id] = ChatMessageHistory() 
    
    return Store[session_id] 

with_message_history =RunnableWithMessageHistory(model , get_session_history)

In [ ]:
config={"configurable":{"session_id":"chat1"}}

In [ ]:
response = with_message_history.invoke( 
           [HumanMessage(content="Hi My name is Jay and I am Data Scientist")]  , 
           config=config ) 


response.content

In [ ]:
response = with_message_history.invoke( 
           [HumanMessage(content="What is my name and what  i am ?")]  , 
           config=config ) 

response.content

In [ ]:
config={"configurable":{"session_id":"chat2"}} 

# here we change the session_id 

In [ ]:
response = with_message_history.invoke( 
           [HumanMessage(content="What is my name and what  i am ?")]  , 
           config=config ) 

response.content 

# see the differnce 

In [ ]:
response = with_message_history.invoke( 
           [HumanMessage(content="Hey my name is Narendra Modi iam an Software engineer?")]  , 
           config=config ) 

# response.content

In [ ]:
response = with_message_history.invoke( 
           [HumanMessage(content="What is my name and what  i am ?")]  , 
           config=config ) 

response.content 

In [ ]:
#Prompt_templates :

In [ ]:
from langchain_core.prompts import ChatPromptTemplate ,  MessagesPlaceholder 

prompt = ChatPromptTemplate.from_messages([
    ("system","You are the helpfull assitance Anser all the Question to best of your knowledge") ,
    MessagesPlaceholder(variable_name="message")
    ] )


chain = prompt | model 

In [ ]:
chain.invoke({"message":[HumanMessage(content="Hi my name is Jayraj and I am Data scinetist")] })

In [ ]:
with_message_history=RunnableWithMessageHistory(chain ,get_session_history)

In [ ]:
config = {"configurable": {"session_id": "chat3"}}

response = with_message_history.invoke( [HumanMessage(content="hi my name is Iron Man ")]  ,config=config )

response.content 

In [ ]:
response = with_message_history.invoke( [HumanMessage(content="Hey but i dont made up Iron , i am like all human being ")]  ,config=config )

response.content 

In [ ]:
response = with_message_history.invoke( [HumanMessage(content="what is my name  ")]  ,config=config )

response.content 

In [ ]:
response = with_message_history.invoke( [HumanMessage(content="I have Visited america three times , india two times and russia 1 time ")]  ,config=config )

response.content 

In [ ]:
response = with_message_history.invoke( [HumanMessage(content="How many times i visited india ? ")]  ,config=config )

response.content 

In [ ]:
config = {"configurable": {"session_id": "chat4"}}

In [ ]:
response = with_message_history.invoke( [HumanMessage(content="How many times i visited india ? ")]  ,config=config )

response.content 

In [ ]:
#Adding More Complexity 

In [ ]:
prompt = ChatPromptTemplate.from_messages( [
    
    ("system","you are an helpfull Assiatance Answer all questions to the best of your ability in {language}") ,
    MessagesPlaceholder(variable_name="message")
])


chain = prompt | model 

response=chain.invoke({"message":[HumanMessage(content="what is the capital of Russia")] ,"language":"Hindi"}) 

response.content



In [ ]:
with_message_history = RunnableWithMessageHistory( chain , 
                           get_session_history , 
                           input_messages_key="message")

In [ ]:
config = {"configurable": {"session_id": "chat4"}}

In [ ]:
response=with_message_history.invoke({"message":[HumanMessage(content="Hi My name is Donland J Trumph & I am a Teacher")] ,
                                        "language":"hindi"}  , 
                                     config=config)  


response.content

In [ ]:
response=with_message_history.invoke({"message":[HumanMessage(content="I am not an american citizen iam an India")] ,
                                        "language":"hindi"}  , 
                                     config=config)  


response.content

In [ ]:
response=with_message_history.invoke({"message":[HumanMessage(content="what is my name and what is my profession ?")] ,
                                        "language":"hindi"}  , 
                                     config=config)  


response.content